# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global data
    x_axis = []
    for i in range(data['sma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global data
    x_axis = []
    for i in range(data['lma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(row['tick_avg'])    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(row['tick_avg'])
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(row['tick_avg'])    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(row['tick_avg'])
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

## File paths

In [5]:
year = 2019
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [6]:
%%time
#df = pd.read_csv(source_file_path, nrows=1000000)
df = pd.read_csv(source_file_path)
df.head()

Wall time: 20.7 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [7]:
%%time
data = {}
data['number_of_ticks'] = 20
df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████| 1459316/1459316 [05:51<00:00, 4153.51it/s]


Records : 1459316
Wall time: 6min 1s


,tick_avg,spread_avg,tick_sd
0,1.146403,0.000732,0.000036
1,1.146509,0.000655,0.000054
2,1.146389,0.000359,0.000146
3,1.146712,0.000216,0.000038
4,1.146684,0.000273,0.000028


In [8]:
%%time

data['pip_diff'] = 0.00001
data['rsi_window'] = 14
data['rs_max'] = 1e6

data['sma_len'] = 20
data['lma_len'] = 50

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 1.16 s


In [9]:
%%time

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Build sema ---------------------------------------------
    if len(data['ssma_list']) < data['sma_len']:
        before_sma()          
    elif len(data['ssma_list']) == data['sma_len']:
        after_sma() 
        df['sema'][i] = data['sema']
        df['sema_diff'][i]= data['sema_diff']
        
    # Build lema ---------------------------------------------
    if len(data['lsma_list']) < data['lma_len']:
        before_lma()     
    elif len(data['lsma_list']) == data['lma_len']:
        after_lma() 
        df['lema'][i] = data['lema']
        df['lema_diff'][i]= data['lema_diff']    
        
df = df.dropna() 

 21%|██████▏                      | 310127/1459316 [1:49:40<6:46:26, 47.12it/s]


KeyboardInterrupt: 

In [10]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Slopes -----------------------------
#df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope_s)
#df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope_l)

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

## Write data to csv

In [ ]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

## Print Report

In [ ]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)